In [ ]:
import gzip
import json
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix
import torch
import scipy.sparse as sp
import math
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
import torch.optim as optim
import time

In [ ]:
with gzip.open('NCSU-DigIC-GraphData-2023-07-25/xbar/1/xbar.json.gz','rb') as f:
    design = json.loads(f.read().decode('utf-8'))
    
instances = pd.DataFrame(design['instances'])
nets = pd.DataFrame(design['nets'])

conn=np.load('NCSU-DigIC-GraphData-2023-07-25/xbar/1/xbar_connectivity.npz')
A = coo_matrix((conn['data'], (conn['row'], conn['col'])), shape=conn['shape'])
A = A.__mul__(A.T)

def buildBST(array,start=0,finish=-1):
    if finish<0:
        finish = len(array)
    mid = (start + finish) // 2
    if mid-start==1:
        ltl=start
    else:
        ltl=buildBST(array,start,mid)
    
    if finish-mid==1:
        gtl=mid
    else:
        gtl=buildBST(array,mid,finish)
        
    return((array[mid],ltl,gtl))

congestion_data = np.load('NCSU-DigIC-GraphData-2023-07-25/xbar/1/xbar_congestion.npz')
xbst=buildBST(congestion_data['xBoundaryList'])
ybst=buildBST(congestion_data['yBoundaryList'])
demand = np.zeros(shape = [instances.shape[0],])


def getGRCIndex(x,y,xbst,ybst):
    while (type(xbst)==tuple):
        if x < xbst[0]:
            xbst=xbst[1]
        else:
            xbst=xbst[2]
            
    while (type(ybst)==tuple):
        if y < ybst[0]:
            ybst=ybst[1]
        else:
            ybst=ybst[2]
            
    return ybst, xbst


for k in range(instances.shape[0]):
    print(k)
    xloc = instances.iloc[k]['xloc']; yloc = instances.iloc[k]['yloc']
    i,j=getGRCIndex(xloc,yloc,xbst,ybst)
    d = 0 
    for l in list(congestion_data['layerList']): 
        lyr=list(congestion_data['layerList']).index(l)
        d += congestion_data['demand'][lyr][i][j]
    demand[k] = d
        
instances['routing_demand'] = demand

In [ ]:
class GraphConvolution(Module):

    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        support = torch.mm(input, self.weight)
        output = torch.spmm(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'

In [ ]:
class GCNRegressionDynamic(nn.Module):
    def __init__(self, nfeat, nhid, dropout, n_layers):
        super(GCNRegressionDynamic, self).__init__()

        self.gc_layers = nn.ModuleList()
        self.dropout = dropout

        self.gc_layers.append(GraphConvolution(nfeat, nhid))

        for _ in range(n_layers - 2):
            self.gc_layers.append(GraphConvolution(nhid, nhid))

        self.gc_layers.append(GraphConvolution(nhid, 1))
    
    def forward(self, x, adj):
        for layer in self.gc_layers[:-1]:
            x = F.relu(layer(x, adj))
            x = F.dropout(x, self.dropout, training=self.training)

        x = self.gc_layers[-1](x, adj)
        return x.squeeze()


In [ ]:
def normalize(mx):
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

In [ ]:
def train(epoch):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(features, adj)
    loss_train = nn.MSELoss()(output[idx_train], target[idx_train])
    loss_train.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print('Epoch: {:04d}'.format(epoch+1),
                'loss_train: {:.4f}'.format(loss_train.item()),
                'time: {:.4f}s'.format(time.time() - t))
    
def test():
    model.eval()
    output = model(features, adj)
    loss_test = nn.MSELoss()(output[idx_test], target[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()))


In [ ]:
idx_train = range(2766)
idx_test = range(2766, 3952)

idx_train = torch.LongTensor(idx_train)
idx_test = torch.LongTensor(idx_test)

In [ ]:
nfeat = 4 
nhid = 64  
dropout = 0.2
n_layers = 1
model = GCNRegressionDynamic(nfeat, nhid, dropout, n_layers) 
optimizer = optim.Adam(model.parameters(),
                       lr=0.01, weight_decay=5e-4)
criterion = nn.MSELoss()

In [ ]:
features = instances[['xloc', 'yloc', 'cell', 'orient']].to_numpy()
features = sp.csr_matrix(features, dtype=np.float32)
features = normalize(features)
features = torch.FloatTensor(np.array(features.todense()))

target = instances[['routing_demand']].to_numpy()
target = torch.tensor(target, dtype=torch.float32).squeeze()

adj = normalize(sp.csr_matrix(A, dtype=np.float32))
adj = torch.tensor(adj.toarray(), dtype=torch.float32)

In [ ]:
results_df = pd.DataFrame(columns=['Layer', 'Test_MSE', 'Time'])

for i in [1, 2, 3, 4, 5]:
    print("k = {layer}".format(layer=i))
    
    model = GCNRegressionDynamic(nfeat, nhid, dropout, i)
    optimizer = optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
    criterion = nn.MSELoss()

    t_total = time.time()
    for epoch in range(100):
        train(epoch)
    elapsed_time = time.time() - t_total
    print("Optimization Finished!")
    print("Total time elapsed: {:.4f}s".format(elapsed_time))

    model.eval()
    output = model(features, adj)
    loss_test = nn.MSELoss()(output[idx_test], target[idx_test])

    results_df = results_df.append({'Layer': i, 'Test_MSE': loss_test.item(), 'Time': elapsed_time}, ignore_index=True)


In [ ]:
results_df

In [ ]:
class AttentionLayer(Module):
    def __init__(self, in_features, out_features, bias=True):
        super(AttentionLayer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input):
        att_weights = F.softmax(torch.mm(input, self.weight), dim=1)
        output = att_weights * input
        if self.bias is not None:
            return output + self.bias
        else:
            return output

class GCNAttentionDynamic(nn.Module):
    def __init__(self, nfeat, nhid, dropout, n_layers):
        super(GCNAttentionDynamic, self).__init__()

        self.gc_layers = nn.ModuleList()
        self.att_layers = nn.ModuleList()
        self.dropout = dropout

        self.gc_layers.append(GraphConvolution(nfeat, nhid))
        self.att_layers.append(AttentionLayer(nhid, nhid))

        for _ in range(n_layers - 2):
            self.gc_layers.append(GraphConvolution(nhid, nhid))
            self.att_layers.append(AttentionLayer(nhid, nhid))

        self.gc_layers.append(GraphConvolution(nhid, 1))
        self.att_layers.append(AttentionLayer(1, 1))

    def forward(self, x, adj):
        for gc_layer, att_layer in zip(self.gc_layers[:-1], self.att_layers[:-1]):
            x = F.relu(gc_layer(x, adj))
            x = att_layer(x)
            x = F.dropout(x, self.dropout, training=self.training)

        x = self.gc_layers[-1](x, adj)
        x = self.att_layers[-1](x)
        return x.squeeze()


In [ ]:
model = GCNRegressionDynamic(nfeat, nhid, dropout, 1)
optimizer = optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
criterion = nn.MSELoss()

t_total = time.time()
for epoch in range(100):
    train(epoch)
elapsed_time = time.time() - t_total
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(elapsed_time))

model.eval()
output = model(features, adj)
loss_test_GCN = nn.MSELoss()(output[idx_test], target[idx_test])

In [ ]:
model = GCNAttentionDynamic(nfeat, nhid, dropout, 1)
optimizer = optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
criterion = nn.MSELoss()

t_total = time.time()
for epoch in range(100):
    train(epoch)
elapsed_time = time.time() - t_total
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(elapsed_time))

model.eval()
output = model(features, adj)
loss_test_attention = nn.MSELoss()(output[idx_test], target[idx_test])

In [ ]:
loss_test_GCN, loss_test_attention

In [ ]:
with gzip.open('NCSU-DigIC-GraphData-2023-07-25/xbar/2/xbar.json.gz','rb') as f:
    design = json.loads(f.read().decode('utf-8'))

instances = pd.DataFrame(design['instances'])
nets = pd.DataFrame(design['nets'])

conn=np.load('NCSU-DigIC-GraphData-2023-07-25/xbar/2/xbar_connectivity.npz')
A = coo_matrix((conn['data'], (conn['row'], conn['col'])), shape=conn['shape'])
A = A.__mul__(A.T)

congestion_data = np.load('NCSU-DigIC-GraphData-2023-07-25/xbar/2/xbar_congestion.npz')
xbst=buildBST(congestion_data['xBoundaryList'])
ybst=buildBST(congestion_data['yBoundaryList'])
demand = np.zeros(shape = [instances.shape[0],])

for k in range(instances.shape[0]):
    xloc = instances.iloc[k]['xloc']; yloc = instances.iloc[k]['yloc']
    i,j=getGRCIndex(xloc,yloc,xbst,ybst)
    d = 0 
    for l in list(congestion_data['layerList']): 
        lyr=list(congestion_data['layerList']).index(l)
        d += congestion_data['demand'][lyr][i][j]
    demand[k] = d
        
instances['routing_demand'] = demand

In [ ]:
features = instances[['xloc', 'yloc', 'cell', 'orient']].to_numpy()
features = sp.csr_matrix(features, dtype=np.float32)
features = normalize(features)
features = torch.FloatTensor(np.array(features.todense()))

target = instances[['routing_demand']].to_numpy()
target = torch.tensor(target, dtype=torch.float32).squeeze()

adj = normalize(sp.csr_matrix(A, dtype=np.float32))
adj = torch.tensor(adj.toarray(), dtype=torch.float32)

In [ ]:
model.eval()
output = model(features, adj)
nn.MSELoss()(output, target)